In [ ]:
pip install sentence-transformers

In [ ]:
# Allowing colab to access your google drive, as well as reading and writing in it.
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
cd drive/MyDrive/TIS

In [ ]:
# Using the same model as example 1 and 2. Our data is saved in a tsv file
# The tsv file is made by copying the "Comments" column into an excel sheet and export it as a tsv file
from sentence_transformers import SentenceTransformer, util;
import pandas as pd;
model = SentenceTransformer('all-MiniLM-L6-v2')
data_df = pd.read_csv('NuSeds test.tsv', sep='\t')

In [12]:
#Getting comments list without empty space and identical comments
comments_list = [*set(data_df.Comments.tolist())]
#print(len(comments_list))

In [13]:
#Removing comments with multiple empty spaces together
import re
comments_range = range(len(comments_list))
for i in comments_range:
  comments_list[i] = re.sub(' +', ' ', comments_list[i])

#print(len(comments_list))

In [14]:
#Getting unique list
comments_list_unique = [*set(comments_list)]
#print(len(comments_list_unique))

In [15]:
#Encode all sentences
embeddings = model.encode(comments_list_unique)

In [16]:
#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_comments_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_comments_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_comments_combinations = sorted(all_comments_combinations, key=lambda x: x[0], reverse=True)

In [22]:
# Writing our results in txt file due to the massive size.
with open('nused_pair_result.txt', 'w') as f:
    for score, i, j in all_comments_combinations:
        f.write("{} | {} | {:.4f}\n".format(comments_list_unique[i], comments_list_unique[j], cos_sim[i][j]))

# New Section